<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/skipgram_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Last amended: 21st May, 2023
Folders: C:\Users\Ashok\OneDrive\Documents\talkingdata\languagemodeling
         C:\Users\Ashok\OneDrive\Documents\skipgrams

Relevant files:
https://github.com/harnalashok/deeplearning-sequences/blob/main/skipgram_generation.ipynb
  

Ref:
https://ljvmiranda921.github.io/notebook/2021/12/11/word-vectors/#pairs
https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview/part-2-word-vectors
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/skipgrams
https://stackoverflow.com/a/1994012

Objectives:
        i)   To get a skipgram paired sequence

"""

## Install software

In [ ]:
! pip install --upgrade gensim 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Call libraries

In [ ]:
# 1.0 Call libraries
%reset -f
import pandas as pd
import numpy as np

# 1.1 Import module imdb & other keras modules
import tensorflow as tf


# 1.2 API to manipulate sequences of words
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import skipgrams

# 1.3
import gensim
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from collections import Counter

# 1.4
import os
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


In [ ]:
# 1.5 Download stopwords from nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# 1.6 Display multiple commands output from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Useful functions


In [ ]:
# 2.0 Function to clean text (from Kaggle):
def review_to_wordlist( review, remove_stopwords=False, stem = False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # 3. Remove new lines
    review_text = review_text.replace('\n', '')
    # 4. Remove words whose length is less than 3
    #
    # 4. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
      stops = set(stopwords.words("english"))
      words = [w for w in words if not w in stops]
    # 5. Optionally perform stemming (false by default)  
    if stem:
      stemmer = PorterStemmer()
      words = [stemmer.stem(w) for w in words ]
    # 5. Return a list of words
    return(words)


## Mount gdrive

In [ ]:
# 2.1

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# 3.0 We will upload our text file 
#      directly from gdrive:

pathToFolder = "/gdrive/MyDrive/Colab_data_files/skipgram/"

## Read data
Upload text file <i>'football.txt'</i> directly to `/content/` folder of colab virtual machine.

In [ ]:
# 3.1 Read foortball.txt:

tx_data = pd.read_csv(pathToFolder+ "combined.txt", sep = "\t", header = 'infer')
tx_data = pd.read_csv(pathToFolder+ "combinedver1.txt", sep = "\t", header = 'infer')
tx_data.head()

,text
0,"In ordinary language, a crime is an unlawful a..."


In [ ]:
# 3.2 Examine relevant column:
tx_data['text']


0    In ordinary language, a crime is an unlawful a...
Name: text, dtype: object

In [ ]:
# 3.3 Get complete text as a list of one string:

text = list(tx_data['text'])
text

['In ordinary language, a crime is an unlawful act punishable by a state or other authority.[1] The term crime does not, in modern criminal law, have any simple and universally accepted definition,[2] though statutory definitions have been provided for certain purposes.[3] The most popular view is that crime is a category created by law; in other words, something is a crime if declared as such by the relevant and applicable law.[2] One proposed definition is that a crime or offence (or criminal offence) is an act harmful not only to some individual but also to a community, society, or the state ("a public wrong"). Such acts are forbidden and punishable by law.[1][4] The notion that acts such as murder, rape, and theft are to be prohibited exists worldwide.[5] What precisely is a criminal offence is defined by the criminal law of each relevant jurisdiction. While many have a catalogue of crimes called the criminal code, in some common law nations no such comprehensive statute exists. Th

## Process text
Clean text and create word-to-int index

In [ ]:
# 4.0 Clean the text and get tokens:

cleaned_tokens = review_to_wordlist(text[0], remove_stopwords=True)

In [ ]:
# 4.1 Look at tokens and how many total words:
cleaned_tokens[:10]
print()
len(cleaned_tokens)

['ordinary',
 'language',
 'crime',
 'unlawful',
 'act',
 'punishable',
 'state',
 'authority',
 'term',
 'crime']

15343

In [ ]:
# 4.2 No of unique words:

max_vocab = len(set(cleaned_tokens))
max_vocab  # 4044, 4727

4727

In [ ]:
# 5.0 Get a dict of which word occurs
#     how many times:

freq_of_words = Counter(cleaned_tokens)
freq_of_words

In [ ]:
# 5.1 Sort words in order of freq
#     Most freq at the top:

vocab = sorted(freq_of_words,
               key=freq_of_words.get,
               reverse=True)

# 5.1.1
len(vocab)
print("\n=====\n")
vocab   # Most freq at the top; least at the bottom


4727

['football',
 'religion',
 'ball',
 'rules',
 'hockey',
 'game',
 'law',
 'religious',
 'played',
 'rugby',
 'one',
 'religions',
 'also',
 'fruit',
 'first',
 'games',
 'may',
 'many',
 'fruits',
 'called',
 'th',
 'century',
 'crime',
 'world',
 'used',
 'ice',
 'modern',
 'form',
 'known',
 'codes',
 'word',
 'time',
 'include',
 'two',
 'state',
 'people',
 'england',
 'association',
 'players',
 'quantum',
 'various',
 'early',
 'sport',
 'including',
 'english',
 'term',
 'common',
 'however',
 'school',
 'use',
 'criminal',
 'public',
 'crimes',
 'code',
 'system',
 'schools',
 'field',
 'playing',
 'would',
 'american',
 'countries',
 'international',
 'ovary',
 'example',
 'laws',
 'several',
 'often',
 'new',
 'sports',
 'play',
 'life',
 'states',
 'ancient',
 'forms',
 'even',
 'league',
 'culture',
 'found',
 'nature',
 'practices',
 'based',
 'theory',
 'later',
 'e',
 'goal',
 'number',
 'different',
 'beliefs',
 'seeds',
 'individual',
 'social',
 'human',
 'concept',
 

In [ ]:
# 5.2 Get a dict of word and its int label:
#     Most freq word gets transformed to 1:

word_index = {word: ii for ii, word in enumerate(vocab, 1)}

In [ ]:
# 5.3 Here is our word to int index:

word_index

## Get skipgrams
Transform text to int sequence and get skipgram pairs

In [ ]:
# 6.0 Function to map a word list to integers
#     as per mapping in word_index dict:

def seq2int(wordList, word_index):
    return [word_index[x] for x in wordList]

In [ ]:
# 6.1 Here is our int seq:

int_seq = seq2int(cleaned_tokens, word_index)
int_seq[:10]

[677, 246, 23, 2142, 109, 439, 35, 1344, 46, 23]

In [ ]:
skipgrams = []
for i,j in enumerate(int_seq):
  if i < len(int_seq)-9:
    t = int_seq[i],int_seq[i+1],int_seq[i+2],int_seq[i+3],int_seq[i+4],int_seq[i+5],int_seq[i+6], int_seq[i+7],int_seq[i+8],int_seq[i+9]
    skipgrams.append(t)

In [ ]:
skipgrams[:5]

[(677, 246, 23, 2142, 109, 439, 35, 1344, 46, 23),
 (246, 23, 2142, 109, 439, 35, 1344, 46, 23, 27),
 (23, 2142, 109, 439, 35, 1344, 46, 23, 27, 51),
 (2142, 109, 439, 35, 1344, 46, 23, 27, 51, 7),
 (109, 439, 35, 1344, 46, 23, 27, 51, 7, 110)]

In [ ]:
# 6.2 Translate the int_seq to pairs of skipgrams:

from tensorflow.keras.preprocessing.sequence import skipgrams
pairs = skipgrams(
                   int_seq,
                   vocabulary_size = max_vocab,
                   window_size=4,
                   negative_samples=1.0,
                   shuffle=True,
                   categorical=False,
                   sampling_table=None,
                   seed=None
                  )

In [ ]:
# 6.3 Look at pairs:

pairs[0]

## Save processed data
Save skipgram pairs and word-to-index dict to `/content/` folder of virtual machine

In [ ]:
# 7.0 Transform to pandas DataFrame:

data = pd.DataFrame(skipgrams, columns = ["a", "b", "c","d", "e", "f", "g", "h", "i", "j"])
data.to_pickle(pathToFolder + "skipgrams.pkl")
data.head()

,a,b,c,d,e,f,g,h,i,j
0,677,246,23,2142,109,439,35,1344,46,23
1,246,23,2142,109,439,35,1344,46,23,27
2,23,2142,109,439,35,1344,46,23,27,51
3,2142,109,439,35,1344,46,23,27,51,7
4,109,439,35,1344,46,23,27,51,7,110


In [ ]:
for i in list(data.columns):
  print(i,data[i].nunique())

a 4724
b 4725
c 4725
d 4725
e 4724
f 4724
g 4724
h 4725
i 4725
j 4726


In [ ]:
# 7.0 Transform to pandas DataFrame:

data = pd.DataFrame(pairs[0], columns = ["a", "b"])
data.to_pickle(pathToFolder + "seq.pkl")
data.head()

In [ ]:
# 7.1 Save word_to_index dict to a text file:

filehandler = open(pathToFolder + "word_index.txt", 'wt')
data = str(word_index)
filehandler.write(data)

81320

## Read back saved files

In [ ]:
# 8.0 Read back pkl file and dictionary:
seq = pd.read_pickle(pathToFolder + "seq.pkl")
seq.head()

,a,b
0,931,1754
1,534,3492
2,342,30
3,796,3348
4,701,998


In [ ]:
# 8.0 Read back pkl file and dictionary:
skipgrams = pd.read_pickle(pathToFolder + "skipgrams.pkl")
skipgrams.head()

,a,b,c,d,e,f,g,h,i,j
0,677,246,23,2142,109,439,35,1344,46,23
1,246,23,2142,109,439,35,1344,46,23,27
2,23,2142,109,439,35,1344,46,23,27,51
3,2142,109,439,35,1344,46,23,27,51,7
4,109,439,35,1344,46,23,27,51,7,110


In [ ]:
skipgrams.tail()

,a,b,c,d,e,f,g,h,i,j
15329,4723,4724,560,241,2130,4725,2141,40,143,35
15330,4724,560,241,2130,4725,2141,40,143,35,40
15331,560,241,2130,4725,2141,40,143,35,40,4726
15332,241,2130,4725,2141,40,143,35,40,4726,55
15333,2130,4725,2141,40,143,35,40,4726,55,4727


In [ ]:
# 8.1 Read saved dict:

filehandler = open(pathToFolder + "word_index.txt", 'r')
filehandler.read()

"{'football': 1, 'religion': 2, 'ball': 3, 'rules': 4, 'hockey': 5, 'game': 6, 'law': 7, 'religious': 8, 'played': 9, 'rugby': 10, 'one': 11, 'religions': 12, 'also': 13, 'fruit': 14, 'first': 15, 'games': 16, 'may': 17, 'many': 18, 'fruits': 19, 'called': 20, 'th': 21, 'century': 22, 'crime': 23, 'world': 24, 'used': 25, 'ice': 26, 'modern': 27, 'form': 28, 'known': 29, 'codes': 30, 'word': 31, 'time': 32, 'include': 33, 'two': 34, 'state': 35, 'people': 36, 'england': 37, 'association': 38, 'players': 39, 'quantum': 40, 'various': 41, 'early': 42, 'sport': 43, 'including': 44, 'english': 45, 'term': 46, 'common': 47, 'however': 48, 'school': 49, 'use': 50, 'criminal': 51, 'public': 52, 'crimes': 53, 'code': 54, 'system': 55, 'schools': 56, 'field': 57, 'playing': 58, 'would': 59, 'american': 60, 'countries': 61, 'international': 62, 'ovary': 63, 'example': 64, 'laws': 65, 'several': 66, 'often': 67, 'new': 68, 'sports': 69, 'play': 70, 'life': 71, 'states': 72, 'ancient': 73, 'forms'

In [ ]:
#####################

In [ ]:

%reset -f
import bs4
import sys
import requests
import  os
pathToStoreFiles = "/content"
os.chdir(pathToStoreFiles)
os.listdir()
# Your wikipedia page title:
wiki_page = 'Religion'   # Religion, Crime, Hockey, Football
# Process text now
res = requests.get(f'https://en.wikipedia.org/wiki/{wiki_page}' )
res.raise_for_status()
wiki = bs4.BeautifulSoup(res.text,"html.parser")

# open a file named as your wiki page in write mode
with open(wiki_page+".txt", "w", encoding="utf-8") as f:
    for i in wiki.select('p'):
        # write each paragraph to the file
        f.write(i.getText())

os.listdir()


['.config',
 'seq.pkl',
 'football.txt',
 'Hockey.txt',
 'word_index.txt',
 'Crime.txt',
 'sample_data']

1

10

9

13

...

196

483

822

143

['.config',
 'seq.pkl',
 'Religion.txt',
 'football.txt',
 'Hockey.txt',
 'word_index.txt',
 'Crime.txt',
 'sample_data']